In [1]:
import pandas as pd
import os

import requests
import re
import time
import numpy as np
import subprocess
import random

from bs4 import BeautifulSoup

### get game tags

In [11]:
raw_data = []

url = "https://www.ufreegames.com/tags.html"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

for x in soup.find_all('a', class_='taglink', href=True):
    raw_data.append({'tag':x.find('p').text, 'url':"https://www.ufreegames.com" + x['href']})
df_tags = pd.DataFrame(raw_data)

In [12]:
df_tags.head()

,tag,url
0,Adventure Games,https://www.ufreegames.com/games/adventure-gam...
1,Baby Games,https://www.ufreegames.com/games/baby-games.html
2,Barbie Games,https://www.ufreegames.com/games/barbie-games....
3,Batman Games,https://www.ufreegames.com/games/batman-games....
4,Ben10 Games,https://www.ufreegames.com/games/ben10-games.html


### get game titles

In [18]:
raw_data = []

for i, row in df_tags.iterrows():    
    response = requests.get(row['url'])
    soup = BeautifulSoup(response.text, 'html.parser')
    
    games = soup.find('div', class_='content content-home').find_all('div', class_='post')
    for game in games:
        gametitle = game.find('p', class_='post-name').text
        gameurl = game.find('a')['href']
#         print(gametitle, gameurl)
        raw_data.append({'game':gametitle, 'url':"https://www.ufreegames.com"+gameurl, 'tag':row['tag']})
        
df_games = pd.DataFrame(raw_data)
df_games.head()

,game,url,tag
0,Stolen Sword,https://www.ufreegames.com/play/stolen-sword.html,Adventure Games
1,Culmination Armed With Wings,https://www.ufreegames.com/play/culmination-ar...,Adventure Games
2,Belial Chapter 2,https://www.ufreegames.com/play/belial-chapter...,Adventure Games
3,Snail Bob 3,https://www.ufreegames.com/play/snail-bob-3.html,Adventure Games
4,Snail Bob,https://www.ufreegames.com/play/snail-bob.html,Adventure Games


In [34]:
raw_data = []
annotated = set()

for i, row in df_games.iterrows():
    try:
        response = requests.get(row['url'])
        soup = BeautifulSoup(response.text, 'html.parser')
    except:
        print("cannot retrieve", row['url'])
    
    print(row['url'])
    if row['url'] in annotated:
        continue #skip
    try:
        infos = soup.find('ul', class_='more-about-list').find_all('li')
        for info in infos:
            if "Likes" in info.find('b').text:
                likes_perc = int(info.text.split('Likes:')[1].split('%')[0])
            if "Category" in info.find('b').text:
                categories = info.text.split("Category:")[1].split(", ")
            if "Played Total" in info.find('b').text:
                play_count = int(info.text.split("Played Total:")[1])
            if "How To Play" in info.find('b').text:
                how_to = info.text.split("How To Play:")[1]
        raw_data.append({'game':row['game'], 
                         'url':row['url'], 
                         'tag': row['tag'], 
                         'categories': categories, 
                         'like_perc':likes_perc, 
                         'play_count': play_count, 
                         'how_to_play': how_to})
        annotated.add(row['url'])
    except:
        print("cannot retrieve info for game", row['url'])
#     time.sleep(3)

df_game_info = pd.DataFrame(raw_data)
df_game_info.to_csv("./ufreegames_game_info.csv", sep='\t')

https://www.ufreegames.com/play/stolen-sword.html
https://www.ufreegames.com/play/culmination-armed-with-wings.html
https://www.ufreegames.com/play/belial-chapter-2.html
https://www.ufreegames.com/play/snail-bob-3.html
https://www.ufreegames.com/play/snail-bob.html
https://www.ufreegames.com/play/snail-bob-2.html
https://www.ufreegames.com/play/fireboy-and-watergirl-the-forest.html
https://www.ufreegames.com/play/home-sheep-home-2-lost-underground.html
https://www.ufreegames.com/play/transmorpher.html
https://www.ufreegames.com/play/dream-symphony.html
https://www.ufreegames.com/play/hidden-crystals-of-deep-earth.html
https://www.ufreegames.com/play/intrusion.html
https://www.ufreegames.com/play/the-company-of-myself.html
https://www.ufreegames.com/play/pig-nukem.html
https://www.ufreegames.com/play/rainbow.html
https://www.ufreegames.com/play/fancy-pants-adventures.html
https://www.ufreegames.com/play/three-thieving-thugs.html
https://www.ufreegames.com/play/doomsday-shooter.html
http

https://www.ufreegames.com/play/dora-s-haircuts.html
https://www.ufreegames.com/play/baby-dora-s-breakfast.html
https://www.ufreegames.com/play/baby-dora-dentist.html
https://www.ufreegames.com/play/baby-dora-snowboard.html
https://www.ufreegames.com/play/baby-barbie-birthday-party.html
https://www.ufreegames.com/play/baby-barbie-learns-gardening.html
https://www.ufreegames.com/play/baby-barbie-stuffed-friends.html
https://www.ufreegames.com/play/baby-barbie-adopts-a-pet.html
https://www.ufreegames.com/play/baby-barbie-playtime-accident.html
https://www.ufreegames.com/play/baby-barbie-laundry-day.html
https://www.ufreegames.com/play/baby-barbie-picnic-day.html
https://www.ufreegames.com/play/barbie-baby-sitter.html
https://www.ufreegames.com/play/baby-bottle-feeding.html
https://www.ufreegames.com/play/baby-elsa-skating-accident.html
https://www.ufreegames.com/play/fairytale-baby-tinkerbell-caring.html
https://www.ufreegames.com/play/frozen-baby-care.html
https://www.ufreegames.com/pla

https://www.ufreegames.com/play/barbie-beauty-bath.html
https://www.ufreegames.com/play/barbie-s-fruitilicious-facial-full-screen-replay.html
https://www.ufreegames.com/play/barbie-find-the-difference.html
https://www.ufreegames.com/play/barbie-ice-cream-decorating.html
https://www.ufreegames.com/play/baby-barbie-slumber-party.html
https://www.ufreegames.com/play/barbie-fabulous-facial-makeover.html
https://www.ufreegames.com/play/barbie-and-the-secret-door-spot-6-diff.html
https://www.ufreegames.com/play/barbie-thumbelina-mix-up.html
https://www.ufreegames.com/play/barbie-real-cosmetics.html
https://www.ufreegames.com/play/barbie-the-sweetest-halloween.html
https://www.ufreegames.com/play/barbie-halloween-dress-up.html
https://www.ufreegames.com/play/messy-barbie.html
https://www.ufreegames.com/play/baby-barbie-rudolf-injury.html
https://www.ufreegames.com/play/baby-barbie-christmas-prep.html
https://www.ufreegames.com/play/barbie-s-christmas-patchwork-dress.html
https://www.ufreegame

https://www.ufreegames.com/play/mario-egypt-adventure.html
https://www.ufreegames.com/play/mario-trail.html
https://www.ufreegames.com/play/mario-vs-sonic-racing.html
https://www.ufreegames.com/play/sonic-rally.html
https://www.ufreegames.com/play/spongebob-bike-3d.html
https://www.ufreegames.com/play/batman-the-dark-ride.html
https://www.ufreegames.com/play/spiderman-halloween-racing.html
https://www.ufreegames.com/play/ben-10-omniverse-legend-racers.html
https://www.ufreegames.com/play/ben-10-stunt-mania.html
https://www.ufreegames.com/play/ben-10-ultimate-harley.html
https://www.ufreegames.com/play/ben-10-hard-bike.html
https://www.ufreegames.com/play/ben-10-halloween-race.html
https://www.ufreegames.com/play/ben-10-rush-3d.html
https://www.ufreegames.com/play/ben-10-motocross.html
https://www.ufreegames.com/play/biker-vs-zombies.html
https://www.ufreegames.com/play/ben-10-super-stunt-bmx.html
https://www.ufreegames.com/play/patrick-motorbike.html
https://www.ufreegames.com/play/spi

https://www.ufreegames.com/play/minecraft-online.html
https://www.ufreegames.com/play/subway-surfers.html
https://www.ufreegames.com/play/sand-balls.html
https://www.ufreegames.com/play/santa-christmas-adventure.html
https://www.ufreegames.com/play/mario-adventure-world.html
https://www.ufreegames.com/play/piano-tiles-class.html
https://www.ufreegames.com/play/young-santa-crazy-gift.html
https://www.ufreegames.com/play/little-dentist-doctor.html
https://www.ufreegames.com/play/stick-santa-christmas-challenge.html
https://www.ufreegames.com/play/spinner-vs-zombies.html
https://www.ufreegames.com/play/fun-race-3d.html
https://www.ufreegames.com/play/shark-adventure.html
https://www.ufreegames.com/play/need-for-speed.html
https://www.ufreegames.com/play/pou.html
https://www.ufreegames.com/play/helix-crazy-jump-3.html
https://www.ufreegames.com/play/cut-the-rope-classic.html
https://www.ufreegames.com/play/christmas-santas-mission.html
https://www.ufreegames.com/play/christmas-brickout.htm

...

https://www.ufreegames.com/play/cycle-race-2.html
https://www.ufreegames.com/play/coaster-racer-2.html
cannot retrieve https://www.ufreegames.comhttp://www.ufreegames.com/play/rally-stage.html
https://www.ufreegames.comhttp://www.ufreegames.com/play/rally-stage.html
cannot retrieve https://www.ufreegames.comhttp://www.ufreegames.com/play/vehicles-2-level-pack.html
https://www.ufreegames.comhttp://www.ufreegames.com/play/vehicles-2-level-pack.html
cannot retrieve https://www.ufreegames.comhttp://www.ufreegames.com/play/ultimate-dirt-bike-usa.html
https://www.ufreegames.comhttp://www.ufreegames.com/play/ultimate-dirt-bike-usa.html
https://www.ufreegames.com/play/offroaders-2.html
https://www.ufreegames.com/play/neon-race-2.html
https://www.ufreegames.com/play/parking-super-skills-2.html
https://www.ufreegames.com/play/baby-dora-racing-battle.html
https://www.ufreegames.com/play/roller-rider.html
https://www.ufreegames.com/play/earn-to-die.html
https://www.ufreegames.com/play/trucksformer

https://www.ufreegames.com/play/skeleton-king-ghost-land.html
https://www.ufreegames.com/play/hulk-vs.html
https://www.ufreegames.com/play/the-batman-sin-city.html
https://www.ufreegames.com/play/slayer.html
https://www.ufreegames.com/play/pirates-of-the-caribbean-cursed-cave-causade.html
https://www.ufreegames.com/play/ninja-ben-in-winterland.html
https://www.ufreegames.com/play/teen-titans-titanic-ambush.html
https://www.ufreegames.com/play/rage-blade.html
https://www.ufreegames.com/play/ninjago-legend-fighting.html
https://www.ufreegames.com/play/rise-of-the-nindroids.html
https://www.ufreegames.com/play/dragon-age-legends-remix-01.html
https://www.ufreegames.com/play/arcane-weapon.html
https://www.ufreegames.com/play/my-dear-boss-3.html
https://www.ufreegames.com/play/amigo-pancho-2.html
https://www.ufreegames.com/play/zero-gravity.html
https://www.ufreegames.com/play/flying-penguins.html
https://www.ufreegames.com/play/3d-space-hawk.html
https://www.ufreegames.com/play/angry-gran-

https://www.ufreegames.com/play/double-guns.html
https://www.ufreegames.com/play/pou.html
https://www.ufreegames.com/play/little-dentist-doctor.html
https://www.ufreegames.com/play/highway-rider-extreme.html
https://www.ufreegames.com/play/pipe-mania.html
https://www.ufreegames.com/play/tic-tac-toe.html
https://www.ufreegames.com/play/wheelie-bike.html
https://www.ufreegames.com/play/word-cross.html
https://www.ufreegames.com/play/uno-classic.html
https://www.ufreegames.com/play/mario-adventure-world.html
https://www.ufreegames.com/play/monster-truck-madness.html
https://www.ufreegames.com/play/moto-beach-rider.html
https://www.ufreegames.com/play/puzzle-ball.html
https://www.ufreegames.com/play/checker-master.html
https://www.ufreegames.com/play/chess-master.html
https://www.ufreegames.com/play/stickman-ping-pong.html
https://www.ufreegames.com/play/100-balls.html
https://www.ufreegames.com/play/rubiks-cube-3d.html
https://www.ufreegames.com/play/amelia-dress-up.html
https://www.ufree

https://www.ufreegames.com/play/extreme-car-driving-simulator.html
https://www.ufreegames.com/play/kick-the-buddy-forever.html
https://www.ufreegames.com/play/bowmasters.html
https://www.ufreegames.com/play/traffic-run.html
https://www.ufreegames.com/play/parking-master.html
cannot retrieve https://www.ufreegames.comhttp://www.ufreegames.com/play/cars-lightning-speed.html
https://www.ufreegames.comhttp://www.ufreegames.com/play/cars-lightning-speed.html
https://www.ufreegames.com/play/subway-surfers.html
https://www.ufreegames.com/play/mario-adventure.html
https://www.ufreegames.com/play/fun-race-3d.html
https://www.ufreegames.com/play/tank-wars.html
https://www.ufreegames.com/play/offroad-racer.html
https://www.ufreegames.com/play/super-drift-3d.html
https://www.ufreegames.com/play/helix-crazy-jump-3.html
https://www.ufreegames.com/play/miami-gta-simulator-3d.html
https://www.ufreegames.com/play/tom-run.html
https://www.ufreegames.com/play/redboy-and-bluegirl.html
https://www.ufreegam

https://www.ufreegames.com/play/military-combat-3d.html
https://www.ufreegames.com/play/alilg-monster-truck-3d.html
https://www.ufreegames.com/play/monster-truck-3d-reloaded.html
https://www.ufreegames.com/play/monster-truck-3d.html
https://www.ufreegames.com/play/mechwarrior-3d.html
https://www.ufreegames.com/play/desert-squad-3d.html
https://www.ufreegames.com/play/3d-wheels-of-steel.html
https://www.ufreegames.com/play/zombie-battlefield.html
https://www.ufreegames.com/play/earn-to-die.html
https://www.ufreegames.com/play/mass-mayhem-extra-bloody-zombie-apocalypse.html
https://www.ufreegames.com/play/zombie-truck-2.html
https://www.ufreegames.com/play/night-of-zombies.html
https://www.ufreegames.com/play/zombotown.html
https://www.ufreegames.com/play/last-line-of-defense-second-wave.html
https://www.ufreegames.com/play/mad-bombs-2.html
https://www.ufreegames.com/play/zombie-vs-hamster.html
https://www.ufreegames.com/play/zombie-launcher.html
https://www.ufreegames.com/play/zombie-bi

In [35]:
df_game_info

,game,url,tag,categories,like_perc,play_count,how_to_play
0,Stolen Sword,https://www.ufreegames.com/play/stolen-sword.html,Adventure Games,"[Puzzle, Arcade, Hyper Casual, Single Player, ...",88,166205,"To control Stolen Sword, simply use your finge..."
1,Culmination Armed With Wings,https://www.ufreegames.com/play/culmination-ar...,Adventure Games,"[Puzzle, Arcade, Hyper Casual, Single Player, ...",98,176752,"To control Culmination Armed With Wings, simpl..."
2,Belial Chapter 2,https://www.ufreegames.com/play/belial-chapter...,Adventure Games,"[Puzzle, Arcade, Hyper Casual, Single Player, ...",83,177836,"To control Belial Chapter 2, simply use your f..."
3,Snail Bob 3,https://www.ufreegames.com/play/snail-bob-3.html,Adventure Games,"[Puzzle, Arcade, Hyper Casual, Single Player, ...",89,131159,"To control Snail Bob 3, simply use your finger..."
4,Snail Bob,https://www.ufreegames.com/play/snail-bob.html,Adventure Games,"[Puzzle, Arcade, Hyper Casual, Single Player, ...",93,255276,"To control Snail Bob, simply use your finger t..."
...,...,...,...,...,...,...,...
1028,Baymax Vs. Zombies,https://www.ufreegames.com/play/baymax-vs-zomb...,Zombie games,"[Puzzle, Arcade, Hyper Casual, Single Player, ...",84,179652,"To control Baymax Vs. Zombies, simply use your..."
1029,Bang The Zombies,https://www.ufreegames.com/play/bang-the-zombi...,Zombie games,"[Puzzle, Arcade, Hyper Casual, Single Player, ...",82,145554,"To control Bang The Zombies, simply use your f..."
1030,Zombies Vs Penguins 3,https://www.ufreegames.com/play/zombies-vs-pen...,Zombie games,"[Puzzle, Arcade, Hyper Casual, Single Player, ...",84,138852,"To control Zombies Vs Penguins 3, simply use y..."
1031,Left To Die,https://www.ufreegames.com/play/left-to-die.html,Zombie games,"[Puzzle, Arcade, Hyper Casual, Single Player, ...",85,121775,"To control Left To Die, simply use your finger..."
